Data Wrangling Notebook for VertNet Mammal Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import re
import uuid 

Silencing warnings that are unnecessary

In [2]:
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    pass

Import Vertnet Mammal Data

In [3]:
mammal = pd.read_csv("./../Original_Data/no_bats_2020-08-12b.csv")

Clean yearCollected column

In [4]:
# Filling N/As with "Unknown"
mammal["eventdate"] = mammal["eventdate"].fillna("Unknown")

# Create yearCollected Column
mammal = mammal.assign(yearCollected = '')

# Creating event date variable
verbatim_date = mammal['eventdate']

# Establishing vertnet filter
vertnet_date_filter = verbatim_date.str.contains("""IV|0000|September|<|NW|latter|unknown|(MCZ)|(MSU)|present|
                                                    and|;|&|mainly|between|Between|BETWEEN|OR|Unknown|UNKNOWN|
                                                    #|TO|\?|\'|----|19--|No Date|\,|\d{4}-\d{4}|(/n) /d|\d{4}[s]|
                                                    \d{4}\'[S]|1075-07-29|975-07-17|2088|9999|0201|1197|
                                                    1260|4560|1024|1119|1192|1072|1186""")

# Grabbing clean data
verbatim_date_clean= verbatim_date[vertnet_date_filter==False]


# Captures year within string
def year_search(year):
    """Search string for 4 digit number and pass to correct function"""
    if (re.search(r'\d{4}$', year)):
        return year_cleaner_front(year)
    elif (re.search(r'^\d{4}', year)):
        return year_cleaner_back(year)

def year_cleaner_front(year):
    """Isolate the year at the beginning of the string"""
    cleaned_year = year[len(year)-4:len(year)]
    return cleaned_year

def year_cleaner_back(year):
    """Isolate the year at the end of the string"""
    cleaned_year = year[0:4]
    return cleaned_year

mammal["yearCollected"] = verbatim_date_clean.apply(year_search)
mammal["yearCollected"] = mammal["yearCollected"].fillna("Unknown")


Clean up lifeStage

In [5]:
# Fill in NA
mammal["lifestage_cor"] = mammal['lifestage_cor'].fillna("Not Collected")

# Create Filters
adult = mammal['lifestage_cor']=="Adult"
juvenile = mammal['lifestage_cor']=="Juvenile"
ns = mammal['lifestage_cor']=="NS"

# Assign correct terms using filters
mammal['lifestage_cor'][adult] = "adult"
mammal['lifestage_cor'][juvenile] = "juvenile"
mammal['lifestage_cor'][ns] = "Not Collected"

Clean up sex column

In [6]:
# Clean up sex column 
female = mammal['sex']=="female"
male = mammal['sex']=="male"
mammal['sex'][(female == False)&(male==False)]=""

Fill Scientific Names with unknown 

In [7]:
mammal["binomial"]=mammal["binomial"].fillna("Unknown")

Adding additional required GEOME columns

In [8]:
mammal=mammal.assign(samplingProtocol="Unknown")
mammal=mammal.assign(basisOfRecord="PreservedSpecimen")

Create verbatimEventDate column

In [9]:
mammal=mammal.assign(verbatimEventDate = '')
mammal['verbatimEventDate']=mammal['eventdate']

Clean up country column 

In [10]:
#Append countries to verbatim locality column
mammal["locality"] = mammal["locality"] + "," + mammal["country"]

#Read GEOME country list
geome_countries = pd.read_csv("./../Mapping Files/geome_country_list.csv")

country_dictionary = {"United States":"USA", "U S A":"USA", 
                      "Philippine Islands":"Philippines",
                      "Indonesia; Borneo":"Indonesia",
                      "Malaysia; Malaya":"Malaysia",
                      "U.S. Virgin Islands":"Virgin Islands",
                      "Republic of South Africa":"South Africa",
                      "Ivory Coast":"Cote d'Ivoire",
                      "Federated States of Micronesia":"Micronesia",
                      "Lesser Antilles; Grenada":"Grenada",
                      "Indonesia; Java":"Indonesia",
                      "Lesser Antilles; Saint Vincent":"Saint Vincent and the Grenadines",
                      "Lesser Antilles; Barbados":"Barbados",
                      "ST VINCENT":"Saint Vincent and the Grenadines",
                      "Lesser Antilles; Montserrat": "Montserrat",
                      "Indonesia; Sumatra":"Indonesia",
                      "Virgin Islands, US":"Virgin Islands",
                      "Lesser Antilles; Antigua":"Antigua and Barbuda",
                      "England":"United Kingdom",
                      "Republic of Trinidad and Tobago":"Trinidad and Tobago",
                      "Trinidad And Tobago; Trinidad":"Trinidad and Tobago",
                      "COMMONWEALTH OF THE NORTHERN MARIANA ISLANDS":"Northern Mariana Islands",
                      "Congo":"Democratic Republic of the Congo",
                      "Malaysia; Sabah":"Malaysia",
                      "Lesser Antilles; Martinique":"Martinique",
                      "Republic of the Marshall Islands":"Marshall Islands",
                      "Commonwealth of the Bahamas":"Bahamas",
                      "Trinidad & Tabago":"Trinidad and Tobago",
                      "United Kingdom; England":"United Kingdom",
                      "United Kingdom; Scotland":"United Kingdom",
                      "United Kingdom; Wales":"United Kingdom",
                      "Lesser Antilles; Dominica":"Dominica",
                      "Papua, New Guinea":"Papua New Guinea",
                      "People's Republic of China":"China",
                      "SCOTLAND":"United Kingdom"}

def country_correction(country): 
    """Corrects country column to geome specific country list"""
    if country in geome_countries.values:
        return country
    elif country in country_dictionary.keys():
        return country_dictionary[country]
    else:
        country = "Unknown"
        return country 

mammal['country'] = mammal['country'].apply(country_correction)
    
#mammal['country'] = mammal.apply(lambda x: country_correction(x.country, x.ind), axis=1)
#mammal['ind'] = np.arange(len(mammal))

# Fill N/A with unknown
#mammal["country"]=mammal["country"].fillna("Unknown")

Create verbatimElevation Column

In [11]:
string_max = mammal["maximumelevationinmeters"].astype(str)
string_min = mammal["minimumelevationinmeters"].astype(str)
mammal['verbatimElevation']= string_max + "," + string_min

Rearrange columns so that template columns are first, followed by measurement values

In [12]:
# Create column list
cols = mammal.columns.tolist()

# Specify desired columns
cols = ['catalognumber',
        'collectioncode',
        'decimallatitude',
        'decimallongitude',
        'verbatimElevation',
        'institutioncode',
        'verbatimEventDate',
        'locality',
        'samplingProtocol',
        'country',
        'sex',
        'lifestage_cor',
        'binomial',
        'basisOfRecord',
        'yearCollected',
        'body_mass.1.value',
        'ear_length.1.value',
        'hind_foot_length.1.value',
        'tail_length.1.value',
        'total_length.1.value', 
        'body_mass.1.units_inferred',
        'ear_length.1.units_inferred',
        'hind_foot_length.1.units_inferred',
        'tail_length.1.units_inferred',
        'total_length.1.units_inferred',
        'body_mass.1.estimated_value',
        'ear_length.1.estimated_value',
        'hind_foot_length.1.estimated_value',
        'tail_length.1.estimated_value',
        'total_length.1.estimated_value']

# Subset dataframe
mammal = mammal[cols]

Matching template and column terms

In [13]:
# Renaming columns 
mammal = mammal.rename(columns = {'catalognumber': 'catalogNumber',
                                 'collectioncode':'collectionCode',
                                 'decimallatitude':'decimalLatitude',
                                 'decimallongitude':'decimalLongitude',
                                 'maximumelevationinmeters':'maximumElevationInMeters',
                                 'minimumelevationinmeters':'minimumElevationInMeters',
                                 'institutioncode' :'institutionCode',
                                 'locality':'verbatimLocality',
                                 'lifestage_cor':'lifeStage',
                                 'binomial':'scientificName'})

Create materialSampleID which is a UUID for each measurement

In [14]:
mammal=mammal.assign(materialSampleID = '')
mammal['materialSampleID'] = [uuid.uuid4().hex for _ in range(len(mammal.index))]

Create eventID and populate it with materialSampleID

In [15]:
mammal=mammal.assign(eventID = mammal["materialSampleID"])

Add required GEOME column locality after reassigning locality to verbatimLocality

In [16]:
mammal=mammal.assign(locality="Unknown")

Creating long version, first specifiying keep variables, then naming type and value

In [17]:
longVersMammal=pd.melt(mammal,
                      id_vars=['catalogNumber',
                      'collectionCode',
                      'decimalLatitude',
                      'decimalLongitude',
                      'verbatimElevation',
                      'yearCollected',
                      'basisOfRecord',
                      'verbatimEventDate',
                      'institutionCode',
                      'lifeStage',
                      'verbatimLocality',
                      'locality',
                      'samplingProtocol',
                      'country',
                      'sex',
                      'scientificName',
                      'materialSampleID',
                      'eventID',
                      'body_mass.1.units_inferred',
                      'ear_length.1.units_inferred',
                      'hind_foot_length.1.units_inferred',
                      'tail_length.1.units_inferred',
                      'total_length.1.units_inferred',
                      'body_mass.1.estimated_value',
                      'ear_length.1.estimated_value',
                      'hind_foot_length.1.estimated_value',
                      'tail_length.1.estimated_value',
                      'total_length.1.estimated_value'], 
                var_name = 'measurementType',
                value_name = 'measurementValue')

In [18]:
longVersMammal

,catalogNumber,collectionCode,decimalLatitude,decimalLongitude,verbatimElevation,yearCollected,basisOfRecord,verbatimEventDate,institutionCode,lifeStage,...,hind_foot_length.1.units_inferred,tail_length.1.units_inferred,total_length.1.units_inferred,body_mass.1.estimated_value,ear_length.1.estimated_value,hind_foot_length.1.estimated_value,tail_length.1.estimated_value,total_length.1.estimated_value,measurementType,measurementValue
0,28211,Mammals,19.180133,-104.159983,"nan,nan",2001,PreservedSpecimen,1/10/2001,OMNH,Not Collected,...,False,False,False,NaN,NaN,NaN,NaN,NaN,body_mass.1.value,47.1
1,28004,Mammals,19.327833,-103.522667,"nan,nan",2001,PreservedSpecimen,1/4/2001,OMNH,Not Collected,...,False,False,False,NaN,NaN,NaN,NaN,NaN,body_mass.1.value,20.9
2,32114,Mammals,35.864056,-88.676917,"nan,nan",2003,PreservedSpecimen,10/8/2003,OMNH,Not Collected,...,False,False,False,NaN,NaN,NaN,NaN,NaN,body_mass.1.value,78
3,32108,Mammals,35.925722,-88.706333,"nan,nan",2003,PreservedSpecimen,10/2/2003,OMNH,Not Collected,...,False,False,False,NaN,NaN,NaN,NaN,NaN,body_mass.1.value,67
4,32304,Mammals,35.901778,-88.70975,"nan,nan",2003,PreservedSpecimen,10/4/2003,OMNH,Not Collected,...,False,False,False,NaN,NaN,NaN,NaN,NaN,body_mass.1.value,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924335,MR.22295,MR,8.53333,-79.85,"nan,nan",1974,PreservedSpecimen,1/26/1974,MSU,Not Collected,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total_length.1.value,NaN
2924336,MR.19051,MR,-37.7814,-58.244,"nan,nan",1970,PreservedSpecimen,4/19/1970,MSU,Not Collected,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total_length.1.value,NaN
2924337,MR.2660,MR,46.7999,-89.301,"nan,nan",1957,PreservedSpecimen,4/15/1957,MSU,Not Collected,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total_length.1.value,NaN
2924338,MR.29308,MR,-20.345,-46.545,"nan,nan",1979,PreservedSpecimen,4/28/1979,MSU,Not Collected,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,total_length.1.value,NaN


If value is inferred by Traiter, add "extracted with traiter; inferred value" to measurementMethod. If the value is estimated by Traiter, add "extracted with Traiter; estimated_value. Else fill with "extracted with Traiter".

In [25]:
# Create measurementMethod column
longVersMammal = longVersMammal.assign(measurementMethod = "Extracted with Traiter")


trait_name_list = ["body_mass.1",
                   "ear_length.1",
                   "hind_foot_length.1",
                   "tail_length.1",
                   "total_length.1"]


def trait_method(trait):
    """
    Adds measurementMethod information based off of "True" values in inferred value
    and estimated value columns
    """

    inferred_column = trait + ".units_inferred"
    estimated_column = trait + ".estimated_value"
    
    inferred_filter = longVersMammal[inferred_column].astype(str).str.contains("TRUE|True|true")
    estimated_filter = longVersMammal[estimated_column].astype(str).str.contains("TRUE|True|true")
    
    longVersMammal["measurementMethod"][inferred_filter] = "Extracted with Traiter ; inferred value"
    longVersMammal["measurementMethod"][estimated_filter] = "Extracted with Traiter ; estimated value"
    longVersMammal["measurementMethod"][estimated_filter & inferred_filter] = "Extracted with Traiter ; estimated value; inferred value"


    
[trait_method(x) for x in trait_name_list]


# Drop inferred and estimated value columns from final dataframe
longVersMammal.drop(columns=['body_mass.1.units_inferred',
                             'ear_length.1.units_inferred',
                             'hind_foot_length.1.units_inferred',
                             'tail_length.1.units_inferred',
                             'total_length.1.units_inferred',
                             'body_mass.1.estimated_value',
                             'ear_length.1.estimated_value',
                             'hind_foot_length.1.estimated_value',
                             'tail_length.1.estimated_value',
                             'total_length.1.estimated_value'])

,catalogNumber,collectionCode,decimalLatitude,decimalLongitude,verbatimElevation,yearCollected,basisOfRecord,verbatimEventDate,institutionCode,lifeStage,...,locality,samplingProtocol,country,sex,scientificName,materialSampleID,eventID,measurementType,measurementValue,measurementMethod
0,28211,Mammals,19.180133,-104.159983,"nan,nan",2001,PreservedSpecimen,1/10/2001,OMNH,Not Collected,...,Unknown,Unknown,Mexico,female,Liomys pictus,c72aefffe3364c3c86114dad0fe7068e,c72aefffe3364c3c86114dad0fe7068e,body_mass.1.value,47.1,Extracted with Traiter
1,28004,Mammals,19.327833,-103.522667,"nan,nan",2001,PreservedSpecimen,1/4/2001,OMNH,Not Collected,...,Unknown,Unknown,Mexico,female,Liomys irroratus,fcd06bdcc089499fafabc751880e784b,fcd06bdcc089499fafabc751880e784b,body_mass.1.value,20.9,Extracted with Traiter
2,32114,Mammals,35.864056,-88.676917,"nan,nan",2003,PreservedSpecimen,10/8/2003,OMNH,Not Collected,...,Unknown,Unknown,USA,male,Sigmodon hispidus,27a680b471ae4222b04e19f5650b4f10,27a680b471ae4222b04e19f5650b4f10,body_mass.1.value,78,Extracted with Traiter
3,32108,Mammals,35.925722,-88.706333,"nan,nan",2003,PreservedSpecimen,10/2/2003,OMNH,Not Collected,...,Unknown,Unknown,USA,male,Sigmodon hispidus,b244fc51b208499eb6e977d85e34b28e,b244fc51b208499eb6e977d85e34b28e,body_mass.1.value,67,Extracted with Traiter
4,32304,Mammals,35.901778,-88.70975,"nan,nan",2003,PreservedSpecimen,10/4/2003,OMNH,Not Collected,...,Unknown,Unknown,USA,female,Peromyscus leucopus,75de7e2108384529ab1ed5414c1f0668,75de7e2108384529ab1ed5414c1f0668,body_mass.1.value,19,Extracted with Traiter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924335,MR.22295,MR,8.53333,-79.85,"nan,nan",1974,PreservedSpecimen,1/26/1974,MSU,Not Collected,...,Unknown,Unknown,Panama,male,Isthmomys pirrensis,64b057c6f1a34528b502bcebff09a544,64b057c6f1a34528b502bcebff09a544,total_length.1.value,NaN,Extracted with Traiter
2924336,MR.19051,MR,-37.7814,-58.244,"nan,nan",1970,PreservedSpecimen,4/19/1970,MSU,Not Collected,...,Unknown,Unknown,Argentina,female,Oxymycterus rufus,a14f7fe37e9e4e0f981073fb410b0a69,a14f7fe37e9e4e0f981073fb410b0a69,total_length.1.value,NaN,Extracted with Traiter
2924337,MR.2660,MR,46.7999,-89.301,"nan,nan",1957,PreservedSpecimen,4/15/1957,MSU,Not Collected,...,Unknown,Unknown,USA,male,Lontra canadensis,52f726a685f14c939805085c5697ebc0,52f726a685f14c939805085c5697ebc0,total_length.1.value,NaN,Extracted with Traiter
2924338,MR.29308,MR,-20.345,-46.545,"nan,nan",1979,PreservedSpecimen,4/28/1979,MSU,Not Collected,...,Unknown,Unknown,Brazil,female,Oligoryzomys nigripes,0c9aaf2ee7a04331907f470cc7f30290,0c9aaf2ee7a04331907f470cc7f30290,total_length.1.value,NaN,Extracted with Traiter


In [28]:
longVersMammal["measurementMethod"].unique()

array(['Extracted with Traiter',
       'Extracted with Traiter ; inferred value',
       'Extracted with Traiter ; estimated value',
       'Extracted with Traiter ; estimated value; inferred value'],
      dtype=object)

Matching trait and ontology terms

In [29]:
# Create trait dictionary 
trait_dict = {'body_mass.1.value':'body mass',
              'ear_length.1.value': 'ear length to notch',
              'hind_foot_length.1.value':'pes length',
              'tail_length.1.value':'tail length',
              'total_length.1.value':'body length'}

def trait_rename(trait): 
    """
    Renames trait names with trait dictionary
    """
    if trait in trait_dict.keys():
        return trait_dict[trait]

longVersMammal['measurementType'] = longVersMammal['measurementType'].apply(trait_rename)

Populating measurementUnit column with appropriate measurement units in long version

In [30]:
# Create measurementUnit column
longVersMammal=longVersMammal.assign(measurementUnit="")

#Create filters
long_body_mass_filter=longVersMammal['measurementType']=="body mass"
long_no_body_filter=longVersMammal['measurementType']!="body mass"

#Assign units using filters
longVersMammal['measurementUnit'][long_body_mass_filter] = "g"
longVersMammal['measurementUnit'][long_no_body_filter] = "mm"


Create diagnosticID which is a unique number for each measurement

In [31]:
longVersMammal=longVersMammal.assign(diagnosticID = '')
longVersMammal['diagnosticID'] = np.arange(len(longVersMammal))

If measurement value equals N/A, delete entire row. Drop range values. 

In [32]:
#Drop N/A
longVersMammal = longVersMammal.dropna(subset=['measurementValue'])

#Drop Range Values
range_value_filter=longVersMammal['measurementValue'].str.contains(",|one", na=False)
longVersMammal['measurementValue'][range_value_filter] = float("nan")
longVersMammal = longVersMammal.dropna(subset=['measurementValue'])

Breaking up the data into more managable sizes for validation and DE storage

In [33]:
# Create chunks list
chunks = []

# Separating files into chunks of ~500,000
chunks = np.array_split(longVersMammal, 6)

Creating data chunks

In [34]:
for i in range(len(chunks)):
    new=i+1
    chunks[i].to_csv('../Mapped_Data/FuTRES_Mammals_VertNet_Global_Modern_'+ str(new) +'.csv', index=False)
    print("mapped_data",i, " done")

mapped_data 0  done
mapped_data 1  done
mapped_data 2  done
mapped_data 3  done
mapped_data 4  done
mapped_data 5  done
